# This notebook implements all the experimentats

In [3]:
# Automatically re-import files when updated
%load_ext autoreload
%autoreload 2  
# Load packages
from models import Inception, downsampler, upsampler
from utils import *
from train import run
from evaluate import evaluate
from matplotlib import pyplot as plt
import time, json, pandas as pd, IPython
from itertools import product

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:CPU:0',)
The dtype policy mixed_float16 may run slowly because this machine does not have a GPU. Only Nvidia GPUs with compute capability of at least 7.0 run quickly with mixed_float16.
If you will use compatible GPU(s) not attached to this host, e.g. by running a multi-worker model, you can ignore this warning. This message will only be logged once


# Run and evaluate

In [23]:
# Define parameters to be tested
params = {
    "adv_attack":[fgsm, ifgsm, carlini_wagner],
    "epsilon":[tf.constant(0.01), tf.constant(0.1)],
    "datasets":[tf.keras.datasets.mnist.load_data(), tf.keras.datasets.fashion_mnist.load_data(), tf.keras.datasets.cifar10.load_data()]
}
params = product(*params.values())

# Loop over parameters
results = {}
for i, (adv_attack, epsilon, ((X_train, y_train), (X_test, y_test))) in enumerate(params):
    # Convert data to tf dataset 
    data_train = dataset_from_arrays(X_train, y_train)
    data_test = dataset_from_arrays(X_test, y_test)

    # Train the models and save them
    run(data_train, adv_attack, epsilon)

    # Load models
    with tf.keras.utils.custom_object_scope({
        'Inception': Inception,
        "downsampler": downsampler,
        "upsampler": upsampler
        }):
        decoder = tf.keras.models.load_model("decoder.keras")
        encoder = tf.keras.models.load_model("encoder.keras")
        classifier = tf.keras.models.load_model("classifier.keras")

    # Evaluate and save
    accuracies = evaluate(data_test, X_train/BATCH_SIZE, encoder, decoder, classifier, adv_attack)
    results[i] = accuracies
    json.dump(results, open("results.json", mode="w"))